## API Tiingo - Diario
###### Helga Zambrana - Abril 2024

#### Importar las librerías necesarias

In [1]:
import requests
import time
from decouple import config
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime, func
from psycopg2 import extras
import datetime

#### Leer el usuario, la contraseña y el token desde la variable de entorno

In [2]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Definir la clase ListaSimbolos para manejar los símbolos

In [3]:
class ListaSimbolos:
    def __init__(self, simbolos):
        self.simbolos = simbolos
    
    def __call__(self):
        return self.simbolos

#### Definir la clase TiingoData para manejar los datos de Tiingo

In [4]:
class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def get_data(self, lista_simbolos):
        simbolos = lista_simbolos()
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices" # Recorrer la lista de símbolos y hacer una solicitud a la API para cada uno
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos para asegurarse de que es una lista de diccionarios
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Aviso: Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                if respuesta.status_code == 429:
                    print(f"Aviso: Demasiadas solicitudes a la API para el símbolo {simbolo}. Esperando 2 segundos...")
                    time.sleep(2)  # Esperar 2 segundos para evitar el límite de solicitudes
                else:
                    print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'fecha',
                'ticker': 'simbolo',
                'open': 'apertura',
                'high': 'maximo',
                'low': 'minimo',
                'close': 'cierre',
                'volume': 'volumen',
            }
            df.rename(columns=columnas, inplace=True)
            # Convertir la columna de fecha a tipo datetime
            df['fecha'] = pd.to_datetime(df['fecha'])
            print("Datos obtenidos exitosamente.")
            print(df)
            return df
        else:
            print("No se obtuvieron datos debido a demasiadas solicitudes a la API.")
            return None

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [18]:
class BaseDatosRedshift:
    def __init__(self, host, port, database_name, user, password):
        self.host = host
        self.port = port
        self.database_name = database_name
        self.user = user
        self.password = password
        self.conn = None
        self.cur = None

    def conectar(self):
        try:
            self.conn = psycopg2.connect(
                dbname=self.database_name,
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port
            )
            self.cur = self.conn.cursor()
            print("Conexión a la base de datos exitosa.")
        except Exception as e:
            print(f"Error al crear la conexión a la base de datos:", e)

    def crear_tabla_tiingo(self):
        if self.conn is not None:
            try:
                # Crear tabla principal si no existe
                if not self.tabla_existe('tiingo'):
                    self.cur.execute("""
                        CREATE TABLE tiingo (
                            simbolo VARCHAR(10) NOT NULL,
                            fecha TIMESTAMP NOT NULL,
                            apertura FLOAT,
                            maximo FLOAT,
                            minimo FLOAT,
                            cierre FLOAT,
                            volumen INT,
                            fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                            PRIMARY KEY (simbolo, fecha)
                        )
                    """)
                    self.conn.commit()
                    print("Tabla tiingo creada exitosamente.")
                
                # Crear tabla temporal si no existe
                if not self.tabla_existe('tiingo_tmp'):
                    self.cur.execute("""
                        CREATE TEMP TABLE tiingo_tmp (
                            simbolo VARCHAR(10),
                            fecha TIMESTAMP,
                            apertura FLOAT,
                            maximo FLOAT,
                            minimo FLOAT,
                            cierre FLOAT,
                            volumen INT
                        )
                    """)
                    self.conn.commit()
                    print("Tabla temporal tiingo_tmp creada exitosamente.")
            except Exception as e:
                print(f"Error al crear la tabla:", e)

    def tabla_existe(self, nombre_tabla):
        self.cur.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = %s)", (nombre_tabla,))
        return self.cur.fetchone()[0]

    def cargar_datos_tiingo(self, df):
        if self.conn is not None:
            try:
                # Convertir DataFrame a lista de tuplas
                data = [tuple(row) for row in df[['simbolo', 'fecha', 'apertura', 'maximo', 'minimo', 'cierre', 'volumen']].values]

                # Insertar datos en la tabla temporal
                insert_query_temp = """
                INSERT INTO tiingo_tmp (simbolo, fecha, apertura, maximo, minimo, cierre, volumen)
                VALUES (%s, %s, %s, %s, %s, %s, %s);
                """

                # Preparar consulta de borrado en tabla principal basada en datos de la tabla temporal
                delete_query = """
                DELETE FROM tiingo
                USING tiingo_tmp
                WHERE tiingo.simbolo = tiingo_tmp.simbolo AND tiingo.fecha = tiingo_tmp.fecha;
                """

                # Insertar los datos actualizados desde la tabla temporal a la tabla principal
                insert_query = """
                INSERT INTO tiingo (simbolo, fecha, apertura, maximo, minimo, cierre, volumen)
                SELECT simbolo, fecha, apertura, maximo, minimo, cierre, volumen FROM tiingo_tmp;
                """

                self.conn.autocommit = False  # Deshabilitar autocommit para comenzar la transacción

                try:
                    # Insertar en tabla temporal
                    self.cur.executemany(insert_query_temp, data)
                    self.conn.commit()

                    # Borrar registros existentes en tabla principal que coinciden con la tabla temporal
                    self.cur.execute(delete_query)
                    self.conn.commit()

                    # Insertar registros actualizados desde la tabla temporal
                    self.cur.execute(insert_query)
                    self.conn.commit()

                    # Limpieza de la tabla temporal
                    self.cur.execute("DELETE FROM tiingo_tmp;")
                    self.conn.commit()

                except Exception as e:
                    self.conn.rollback()  # Revertir transacción en caso de error
                    raise e

                finally:
                    self.conn.autocommit = True  # Restablecer autocommit

                print("Datos cargados y actualizados correctamente en 'tiingo'.")
            except Exception as e:
                print(f"Error al cargar los datos en la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

#### Obtener datos de Tiingo

In [6]:
# Lista de símbolos a consultar
simbolos = ['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA']
lista_simbolos = ListaSimbolos(simbolos)

# Crear instancia de TiingoData
tiingo_data = TiingoData(token)

# Obtener y mostrar los datos de Tiingo en un DataFrame de pandas
datos_tiingo_df = tiingo_data.get_data(lista_simbolos)

Datos obtenidos exitosamente.
                         fecha  cierre  maximo    minimo  apertura    volumen  \
0    2023-01-03 00:00:00+00:00   85.82   86.96   84.2050     85.46   76706040   
1    2023-01-04 00:00:00+00:00   85.14   86.98   83.3600     86.55   68885123   
2    2023-01-05 00:00:00+00:00   83.12   85.42   83.0700     85.33   67930825   
3    2023-01-06 00:00:00+00:00   86.08   86.40   81.4300     83.03   83303361   
4    2023-01-09 00:00:00+00:00   87.36   89.48   87.0800     87.46   65266056   
...                        ...     ...     ...       ...       ...        ...   
3295 2024-04-19 00:00:00+00:00  147.05  150.94  146.2200    148.97   87074500   
3296 2024-04-22 00:00:00+00:00  142.05  144.44  138.8025    140.56  107097564   
3297 2024-04-23 00:00:00+00:00  144.68  147.26  141.1100    143.33  124545104   
3298 2024-04-24 00:00:00+00:00  162.13  167.97  157.5100    162.84  181178020   
3299 2024-04-25 00:00:00+00:00  170.18  170.88  158.3600    158.96  126427521  

#### Cargar datos en Redshift

In [19]:
# Crear instancia de la clase BaseDatosRedshift con los datos de conexión
try:
    bd_redshift = BaseDatosRedshift(my_host, port, database_name, user, password)
    bd_redshift.conectar()

    # Crear tabla en la base de datos
    bd_redshift.crear_tabla_tiingo()

    # Cargar los datos en Redshift si hay datos disponibles
    if datos_tiingo_df is not None and not datos_tiingo_df.empty:
        bd_redshift.cargar_datos_tiingo(datos_tiingo_df)
    else:
        print("No se obtuvieron datos para los símbolos especificados o el DataFrame está vacío.")

except Exception as e:
    print(f"Ha ocurrido un error durante la operación: {e}")
finally:
    if bd_redshift.conn is not None:
        bd_redshift.conn.close()
        print("Conexión cerrada.")

Conexión a la base de datos exitosa.
Tabla temporal tiingo_tmp creada exitosamente.
Datos cargados y actualizados correctamente en 'tiingo'.
Conexión cerrada.
